- 如何评价一个模型的输出是不是瞎猜：PPL 指标；
- 重新 review GPT 的过程
    - input_ids: 1*1024, 一个（bs）长度为 1024 的 token ids
    - last_hidden_states: 1\*1024\*768
        - last layer hidden states of (transformer)
    - lm_logits: 1\*1024\*50257
        - lm head，将每一个位置上的 token 的 hidden state，映射到整个词表维度上的概率分布输出
- shift labels 与损失计算

    ```
    labels = labels.to(lm_logits.device)
    
    # Shift so that tokens < n predict n
    shift_logits = lm_logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    # Flatten the tokens
    loss_fct = CrossEntropyLoss()
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ```

In [4]:
logits = [1, 2, 3, 4, 5]
labels = [1, 2, 3, 4, 5]
print(logits, logits[:-1])
print(labels, labels[1:])

[1, 2, 3, 4, 5] [1, 2, 3, 4]
[1, 2, 3, 4, 5] [2, 3, 4, 5]


## casual/decoder only 单向注意力的实现


- modeling_gpt2.py
    - GPT2Attention._attn

```
if not self.is_cross_attention:
    # if only "normal" attention layer implements causal mask
    query_length, key_length = query.size(-2), key.size(-2)
    causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length]
    mask_value = torch.finfo(attn_weights.dtype).min
    # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
    # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
    mask_value = torch.full([], mask_value, dtype=attn_weights.dtype, device=attn_weights.device)
    attn_weights = torch.where(causal_mask, attn_weights.to(attn_weights.dtype), mask_value)

if attention_mask is not None:
    # Apply the attention mask
    attn_weights = attn_weights + attention_mask
```